In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/suvidha-project/x_test.pkl
/kaggle/input/suvidha-project/y_train.pkl
/kaggle/input/suvidha-project/x_tokenizer.pkl
/kaggle/input/suvidha-project/__results__.html
/kaggle/input/suvidha-project/x_val.pkl
/kaggle/input/suvidha-project/__resultx__.html
/kaggle/input/suvidha-project/x_train.pkl
/kaggle/input/suvidha-project/test_data.pkl
/kaggle/input/suvidha-project/__notebook__.ipynb
/kaggle/input/suvidha-project/__output__.json
/kaggle/input/suvidha-project/y_test.pkl
/kaggle/input/suvidha-project/y_val.pkl
/kaggle/input/suvidha-project/train_data.pkl
/kaggle/input/suvidha-project/y_tokenizer.pkl
/kaggle/input/suvidha-project/val_data.pkl
/kaggle/input/suvidha-project/custom.css
/kaggle/input/suvidha-project/final_gru_model/fingerprint.pb
/kaggle/input/suvidha-project/final_gru_model/saved_model.pb
/kaggle/input/suvidha-project/final_gru_model/keras_metadata.pb
/kaggle/input/suvidha-project/final_gru_model/variables/variables.index
/kaggle/input/suvidha-project/final_gru_mo

In [2]:
import numpy as np
import pandas as pd
import re
import os
import pickle
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate, GRU
from tensorflow.keras.callbacks import ModelCheckpoint

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
!pip install rouge
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# # Load datasets
# train_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')
# validate_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')
# test_data = pd.read_csv('/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')

# # Data cleaning function
# def clean_text(text):
#     text = text.lower()
#     text = re.sub(r'\s+', ' ', text)
#     text = re.sub(r"[^a-zA-Z0-9]", " ", text)
#     return text

# # Preprocess the data
# train_data['cleaned_article'] = train_data['article'].apply(clean_text)
# train_data['cleaned_highlights'] = train_data['highlights'].apply(clean_text)

# validate_data['cleaned_article'] = validate_data['article'].apply(clean_text)
# validate_data['cleaned_highlights'] = validate_data['highlights'].apply(clean_text)

# test_data['cleaned_article'] = test_data['article'].apply(clean_text)
# test_data['cleaned_highlights'] = test_data['highlights'].apply(clean_text)

In [5]:
# # saving data pickle
# train_data.to_pickle('/kaggle/input/suvidha-project/train_data.pkl')
# validate_data.to_pickle('/kaggle/input/suvidha-project/val_data.pkl')
# test_data.to_pickle('/kaggle/input/suvidha-project/test_data.pkl')

In [6]:
# And to load:
train_data = pd.read_pickle('/kaggle/input/suvidha-project/train_data.pkl')
validate_data = pd.read_pickle('/kaggle/input/suvidha-project/val_data.pkl')
test_data = pd.read_pickle('/kaggle/input/suvidha-project/test_data.pkl')

In [7]:
train_data.info()  # Check data types and look for anything unusual

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   id                  287113 non-null  object
 1   article             287113 non-null  object
 2   highlights          287113 non-null  object
 3   cleaned_article     287113 non-null  object
 4   cleaned_highlights  287113 non-null  object
dtypes: object(5)
memory usage: 11.0+ MB


In [8]:
# Define maximum sequence lengths based on dataset summary
max_len_article = 800  # Adjusted for the dataset
max_len_highlights = 60  # Adjusted for the dataset

In [9]:
# Tokenize the text
x_tokenizer = Tokenizer()
y_tokenizer = Tokenizer()

# Fit the tokenizers on the training data
x_tokenizer.fit_on_texts(list(train_data['cleaned_article']))
y_tokenizer.fit_on_texts(list(train_data['cleaned_highlights']))

# Convert texts to sequences and pad them
def convert_to_sequences(data, tokenizer, max_len):
    sequences = tokenizer.texts_to_sequences(data)
    padded = pad_sequences(sequences, maxlen=max_len, padding='post')
    return padded

In [10]:
# Save Pickle the tokenizers
# import pickle
# with open('/kaggle/input/suvidha-project/x_tokenizer.pkl', 'wb') as handle:
#     pickle.dump(x_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('/kaggle/input/suvidha-project/y_tokenizer.pkl', 'wb') as handle:
#     pickle.dump(y_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
# Load the tokenizers
import pickle

with open('/kaggle/input/suvidha-project/x_tokenizer.pkl', 'rb') as handle:
    x_tokenizer = pickle.load(handle)

with open('/kaggle/input/suvidha-project/y_tokenizer.pkl', 'rb') as handle:
    y_tokenizer = pickle.load(handle)

In [12]:
# x_train = convert_to_sequences(train_data['cleaned_article'], x_tokenizer, max_len_article)
# y_train = convert_to_sequences(train_data['cleaned_highlights'], y_tokenizer, max_len_highlights)

# x_val = convert_to_sequences(validate_data['cleaned_article'], x_tokenizer, max_len_article)
# y_val = convert_to_sequences(validate_data['cleaned_highlights'], y_tokenizer, max_len_highlights)

# x_test = convert_to_sequences(test_data['cleaned_article'], x_tokenizer, max_len_article)
# y_test = convert_to_sequences(test_data['cleaned_highlights'], y_tokenizer, max_len_highlights)

In [13]:
# Assuming x_tokenizer and y_tokenizer are already fitted on the articles and highlights respectively
# Define vocabulary sizes
x_voc_size = len(x_tokenizer.word_index) + 1
y_voc_size = len(y_tokenizer.word_index) + 1

In [14]:
# import pickle

# # Save the tokenized and padded sequences
# with open('/kaggle/input/suvidha-project/x_train.pkl', 'wb') as file:
#     pickle.dump(x_train, file)
# with open('/kaggle/input/suvidha-project/y_train.pkl', 'wb') as file:
#     pickle.dump(y_train, file)
# with open('/kaggle/input/suvidha-project/x_val.pkl', 'wb') as file:
#     pickle.dump(x_val, file)
# with open('/kaggle/input/suvidha-project/y_val.pkl', 'wb') as file:
#     pickle.dump(y_val, file)
# with open('/kaggle/input/suvidha-project/x_test.pkl', 'wb') as file:
#     pickle.dump(x_test, file)
# with open('/kaggle/input/suvidha-project/y_test.pkl', 'wb') as file:
#     pickle.dump(y_test, file)

In [15]:
# Load the tokenized and padded sequences

import pickle

with open('/kaggle/input/suvidha-project/x_train.pkl', 'rb') as file:
    x_train = pickle.load(file)
with open('/kaggle/input/suvidha-project/y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)
with open('/kaggle/input/suvidha-project/x_val.pkl', 'rb') as file:
    x_val = pickle.load(file)
with open('/kaggle/input/suvidha-project/y_val.pkl', 'rb') as file:
    y_val = pickle.load(file)
with open('/kaggle/input/suvidha-project/x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)
with open('/kaggle/input/suvidha-project/y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)

# LSTM SEQ2SEQ

In [16]:
# Encoder
encoder_inputs = Input(shape=(max_len_article,))
enc_emb = Embedding(x_voc_size, 100)(encoder_inputs)
encoder_lstm = Bidirectional(LSTM(100, return_state=True))
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(enc_emb)
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(y_voc_size, 100)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(y_voc_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [18]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.indices = np.arange(len(self.x))

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[inds]
        batch_y = self.y[inds]
        return [batch_x, batch_y], batch_y

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

In [19]:
# Assuming x_train, y_train, x_val, y_val are your preprocessed data
batch_size = 64  # Define your batch size
train_gen = DataGenerator(x_train, y_train, batch_size)
val_gen = DataGenerator(x_val, y_val, batch_size)


In [20]:
# Set up the checkpoint directory and file
checkpoint_path = "/kaggle/input/suvidha-project/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Ensure the checkpoint directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch'
)

In [21]:
# history = model.fit(
#     train_gen,
#     epochs=5,
#     validation_data=val_gen,
#     callbacks=[checkpoint_callback]
# )

In [22]:
# model.save('/kaggle/input/suvidha-project/cnn_dailymail/seq2seq_model.h5')

In [23]:
# test_loss = model.evaluate([x_test, y_test], y_test)
# print(f'Test Loss: {test_loss}')


In [24]:
from tensorflow.keras.models import load_model

# Correct the file path by adding quotes
model_file_path = '/kaggle/input/suvidha-project/cnn_dailymail/seq2seq_model.h5'

# Load the saved model
loaded_model = load_model(model_file_path)

# Verify the model's structure
loaded_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 800)]                0         []                            
                                                                                                  
 embedding_6 (Embedding)     (None, 800, 100)             4761990   ['input_7[0][0]']             
                                                          0                                       
                                                                                                  
 input_8 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, 200),                160800    ['embedding_6[0][0]']   

In [25]:
# Assuming your model's encoder and decoder LSTM units have 200 dimensions
latent_dim = 200

# Redefine the encoder model to output the states
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)


In [26]:
reverse_target_word_index = y_tokenizer.index_word
reverse_source_word_index = x_tokenizer.word_index
target_word_index = y_tokenizer.word_index

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_word_index.get('starttoken', 1)

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index in reverse_target_word_index:
            sampled_word = reverse_target_word_index[sampled_token_index]
        else:
            sampled_word = 'UNK'  # Using 'UNK' for unknown words

        if sampled_word == 'endtoken' or len(decoded_sentence) > max_len_article:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence



In [27]:
# import random

# # Define the number of samples to decode
# num_samples_to_decode = 50  # Adjust this number as needed

# # Randomly select indices for the subset
# random_indices = random.sample(range(len(x_test)), min(num_samples_to_decode, len(x_test)))

# # Generate and save decoded sentences and original summaries for the random subset
# decoded_sentences = []
# original_summaries = []

# for i in random_indices:
#     input_seq = x_test[i].reshape(1, -1)
#     decoded_sentence = decode_sequence(input_seq)
#     decoded_sentences.append(decoded_sentence)
#     original_summaries.append(y_test[i])  # Assuming y_test contains the actual summaries

# # ... code to pickle decoded_sentences and original_summaries ...


In [28]:
import pickle

# File paths for saving
decoded_sentences_file = '/kaggle/input/decoded/decoded_sentences.pkl'
original_summaries_file = '/kaggle/input/decoded/original_summaries.pkl'



In [29]:
# # Save decoded sentences
# with open(decoded_sentences_file, 'wb') as file:
#     pickle.dump(decoded_sentences, file)

# # Save original summaries
# with open(original_summaries_file, 'wb') as file:
#     pickle.dump(original_summaries, file)


In [30]:
# Load decoded sentences
with open(decoded_sentences_file, 'rb') as file:
    decoded_sentences = pickle.load(file)

# Load original summaries
with open(original_summaries_file, 'rb') as file:
    original_summaries = pickle.load(file)


In [31]:
# Check the type of the first element (if the list is not empty)
if original_summaries:
    print(f"Type of the first summary: {type(original_summaries[0])}")


Type of the first summary: <class 'numpy.ndarray'>


In [32]:
if decoded_sentences:
    print(f"Type of the first decoded summary: {type(decoded_sentences[0])}")

Type of the first decoded summary: <class 'str'>


In [33]:
def seq_to_text(sequence, tokenizer):
    word_index = tokenizer.index_word
    return ' '.join([word_index.get(i, '?') for i in sequence if i != 0])  # Replace 0 with your tokenizer's padding token index, if different

# Convert original summaries from sequences to text
text_original_summaries = [seq_to_text(summary, y_tokenizer) for summary in original_summaries]


In [34]:
if original_summaries and isinstance(original_summaries[0], bytes):
    original_summaries = [summary.decode('utf-8') for summary in original_summaries]


In [35]:
from nltk.tokenize import word_tokenize
processed_decoded_summaries = [' '.join(word_tokenize(summary)) for summary in decoded_sentences]
processed_original_summaries = [' '.join(word_tokenize(summary)) for summary in text_original_summaries]


In [36]:
print(x_test[0])  # Inspect the first preprocessed test sample
print(y_test[0])  # Inspect the first actual test label



[   381   2570    109    822   2639   1151      2     24    455   2503
      4   2503     15   2540   1343      5     55    337      2      1
   6232     84   1000     28   4213     70    257    161   3140     49
   3231     12   1477   1037     18    630     30    142      9      1
  12441    692     11  32814     12     35     98   5461     14      7
   1477     99    249      4    674      6   2086     51     68  31929
     71      1   1757    901  12441    692     11   3231   1477     99
    249      4    674      6   2086     32    137      3    125      7
   3342   7398    199    213     48     21      1    405      5   3365
     16     18      3    201    771      9     90      1    140     12
    651      2    213   1848     10   1211   1503     11   3231     14
    584     42  34525      3   3024   1188      5    692     10   2647
      6      3     93     94   1211     22     51    520      2    692
      4    418     68   2647     16   2667 162163   3342   3555     11
      

In [37]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Calculate ROUGE scores
rouge = Rouge()
rouge_scores = rouge.get_scores(processed_decoded_summaries, processed_original_summaries, avg=True)

# Calculate BLEU scores
bleu_scores = [sentence_bleu([ref.split()], hyp.split(), smoothing_function=SmoothingFunction().method1)
               for ref, hyp in zip(processed_original_summaries, processed_decoded_summaries)]
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Print the scores
print("ROUGE Scores:", rouge_scores)
print("Average BLEU Score:", avg_bleu_score)


ROUGE Scores: {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0}}
Average BLEU Score: 0.0


In [38]:
for i in range(10):
    print("Generated summary:", decoded_sentences[i])
    print("Original summary:", text_original_summaries[i])
    print("\n")


Generated summary:  pedropsa torkham fuglesang forbidden eurorap comminty comminty chelios twc twc dower persuasive veneman
Original summary: collette dinnigan s paddington 6 million home has hit the market she and her husband bradley cocks paid 4 45 million for it back in 2009 the luxury house will go under the hammer on may 23 the four bedroom two storey sandstone property was built in 1880 the fashionista and her husband have carefully renovated the property


Generated summary:  daringly denunciations majene rennard gonzales zamost exersice innocuous hitchhike hitchhike espen 1979
Original summary: abby has a rare genetic syndrome and has cataracts as a result her mother amanda said she has no vision at all without her glasses felt her daughter had been discriminated against when she found out other children at abby s school got to wear their glasses school photography has since apologized and will re take abby s picture for free


Generated summary:  mcpartlin clarksville eyesee 3

In [39]:
for i in range(10):
    print("Original summary:", original_summaries[i])
    print("Generated summary:", decoded_sentences[i])
    print("\n")


Original summary: [ 32433 110284      7  13969    217    133     68     16    197      1
    902     25      6     20    256   3471  55082    539    195    990
    133      8     22     88      3    664      1   1659    121     28
    223    151      1   4551     10    114    508      1     87   1606
     41   5520  21897    743      9    870      3  26797      1  41016
      6     20    256     24   8524  13080      1    743      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0]
Generated summary:  pedropsa torkham fuglesang forbidden eurorap comminty comminty chelios twc twc dower persuasive veneman


Original summary: [11179    16     5  1122  4218  2506     6    16 31888    21     5  1130
    20   103  2930    35    25    16    73  4107    14

# Multi-gpu

In [40]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input, GRU, Dense, Embedding
# from tensorflow.keras.callbacks import ModelCheckpoint
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# import numpy as np
# import os
# import tensorflow as tf

In [41]:
# # Check available GPUs
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Enable GPU memory growth
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)

#         # Create a MirroredStrategy
#         strategy = tf.distribute.MirroredStrategy()
#         print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

#         # Open a strategy scope
#         with strategy.scope():
#             # Encoder
#             encoder_inputs = Input(shape=(max_len_text,))
#             enc_emb = Embedding(x_voc_size, 100)(encoder_inputs)
#             encoder_lstm = Bidirectional(LSTM(100, return_state=True))
#             encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(enc_emb)
#             state_h = Concatenate()([forward_h, backward_h])
#             state_c = Concatenate()([forward_c, backward_c])
#             encoder_states = [state_h, state_c]

#             # Decoder
#             decoder_inputs = Input(shape=(None,))
#             dec_emb_layer = Embedding(y_voc_size, 100)
#             dec_emb = dec_emb_layer(decoder_inputs)
#             decoder_lstm = LSTM(200, return_sequences=True, return_state=True)
#             decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
#             decoder_dense = Dense(y_voc_size, activation='softmax')
#             decoder_outputs = decoder_dense(decoder_outputs)

#             # Define the model
#             model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
            
#             model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
            
#             from tensorflow.keras.utils import Sequence

#             class DataGenerator(Sequence):
#                 def __init__(self, x_set, y_set, batch_size):
#                     self.x, self.y = x_set, y_set
#                     self.batch_size = batch_size
#                     self.indices = np.arange(len(self.x))

#                 def __len__(self):
#                     return int(np.ceil(len(self.x) / self.batch_size))

#                 def __getitem__(self, idx):
#                     inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
#                     batch_x = self.x[inds]
#                     batch_y = self.y[inds]
#                     return [batch_x, batch_y], batch_y

#                 def on_epoch_end(self):
#                     np.random.shuffle(self.indices)
                    
#             # Assuming x_train, y_train, x_val, y_val are your preprocessed data
#             batch_size = 64  # Define your batch size
#             train_gen = DataGenerator(x_train, y_train, batch_size)
#             val_gen = DataGenerator(x_val, y_val, batch_size)
            
#             # Set up the checkpoint directory and file
#             checkpoint_path = "/kaggle/working/cnn_dailymail/cp.ckpt"
#             checkpoint_dir = os.path.dirname(checkpoint_path)

#             # Ensure the checkpoint directory exists
#             os.makedirs(checkpoint_dir, exist_ok=True)

#             # ModelCheckpoint callback
#             checkpoint_callback = ModelCheckpoint(
#                 filepath=checkpoint_path,
#                 monitor='val_loss',
#                 verbose=1,
#                 save_best_only=True,
#                 save_weights_only=True,
#                 mode='auto',
#                 save_freq='epoch'
#             )
            
#             history = model.fit(
#             train_gen,
#             epochs=5,
#             validation_data=val_gen,
#             callbacks=[checkpoint_callback]
#         )
            
#     except Exception as e:
#         print(e)
# else:
#     print('No GPUs available.')
            

In [42]:
# model.save('/kaggle/working/cnn_dailymail/seq2seq_model.h5')

# GRU****

In [43]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
import tensorflow as tf


# # Define maximum sequence lengths based on dataset summary
# max_len_article = 800  # Adjusted for the dataset
# max_len_highlights = 60  # Adjusted for the dataset

# # Assuming x_tokenizer and y_tokenizer are already fitted on the articles and highlights respectively
# # Define vocabulary sizes
# x_voc_size = len(x_tokenizer.word_index) + 1
# y_voc_size = len(y_tokenizer.word_index) + 1

# Pad the sequences to the max length for articles
x_train_padded = pad_sequences(x_train, maxlen=max_len_article, padding='post')
x_val_padded = pad_sequences(x_val, maxlen=max_len_article, padding='post')

# Add a start token to the beginning of each target sequence and end token at the end
start_token = 1
end_token = 2

# Assuming y_train and y_val already contain end tokens, we need to add start tokens
y_train_with_start = np.hstack([np.full((y_train.shape[0], 1), start_token), y_train])
y_val_with_start = np.hstack([np.full((y_val.shape[0], 1), start_token), y_val])

# Ensure the sequences are of length 100, you might need to truncate or pad accordingly
y_train_padded = pad_sequences(y_train_with_start, maxlen=max_len_highlights, padding='post', truncating='post')
y_val_padded = pad_sequences(y_val_with_start, maxlen=max_len_highlights, padding='post', truncating='post')

# Encoder
encoder_inputs = Input(shape=(max_len_article,))
encoder_embedding = Embedding(input_dim=x_voc_size, output_dim=100)(encoder_inputs)
encoder_gru = GRU(100, return_state=True)
encoder_outputs, encoder_state = encoder_gru(encoder_embedding)

# Decoder
decoder_inputs = Input(shape=(max_len_highlights-1,))  # -1 because we shift the decoder inputs and outputs
decoder_embedding = Embedding(input_dim=y_voc_size, output_dim=100)(decoder_inputs)
decoder_gru = GRU(100, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=encoder_state)
decoder_dense = Dense(y_voc_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [44]:
# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Define the checkpoint callback
checkpoint_path = "/kaggle/working/checkpoints_gru/ckpt_{epoch}"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    verbose=1
)

# Define the checkpoint directory
checkpoint_dir = "/kaggle/working/checkpoints_gru"

# Load the latest checkpoint
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
initial_epoch = 0
if latest_checkpoint:
    print(f"Loading weights from {latest_checkpoint}")
    model.load_weights(latest_checkpoint)

    # Extract the epoch number from the filename
    checkpoint_name = os.path.basename(latest_checkpoint)
    checkpoint_epoch = checkpoint_name.split('_')[-1]
    initial_epoch = int(checkpoint_epoch)
else:
    print("No checkpoint found. Starting training from scratch.")


No checkpoint found. Starting training from scratch.


In [45]:
# # Train the model with the correct shapes
# history = model.fit(
#     [x_train_padded, y_train_padded[:,:-1]],
#     y_train_padded.reshape(y_train_padded.shape[0], y_train_padded.shape[1], 1)[:,1:],
#     initial_epoch=initial_epoch,
#     epochs=5,
#     batch_size=64,
#     validation_data=(
#         [x_val_padded, y_val_padded[:,:-1]],
#         y_val_padded.reshape(y_val_padded.shape[0], y_val_padded.shape[1], 1)[:,1:]
#     ),
#     callbacks=[checkpoint_callback]
# )



In [46]:
# Save the entire model after training
model_save_path = "/kaggle/working/final_gru_model"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /kaggle/working/final_gru_model


In [47]:
import tensorflow as tf

# Path to the saved model directory
model_load_path = "/kaggle/input/suvidha-project/final_gru_model"  # Update with your actual path

# Load the model
loaded_model = tf.keras.models.load_model(model_load_path)
print("Model loaded successfully.")

# Now you can use loaded_model to make predictions, evaluate, etc.


Model loaded successfully.


In [48]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 800)]                0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 59)]                 0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 800, 100)             4761990   ['input_5[0][0]']             
                                                          0                                       
                                                                                                  
 embedding_3 (Embedding)     (None, 59, 100)              1551580   ['input_6[0][0]']       

In [49]:
sample_size = 10 # Try with an even smaller number of samples
x_test_sample = x_val_padded[:sample_size]
y_test_sample = y_val_padded[:sample_size, :-1]

sample_predictions = model.predict([x_test_sample, y_test_sample])


1/1 [==============================] - 1s 1s/step


In [50]:
from tqdm import tqdm

def decode_sequence(input_sequence, tokenizer):
    text = ''
    for i in input_sequence:
        if (i != 0):  # skipping the padding (0)
            text += tokenizer.index_word[i] + ' '
    return text.strip()

# Choose the size of the subset you want to decode
subset_size = 100  # For example, decode only the first 100 sequences

# Assuming y_tokenizer is your tokenizer for the target sequences
# Decode only a subset of the predictions
subset_predictions = np.argmax(sample_predictions, axis=-1)[:subset_size]
predicted_texts = [decode_sequence(p, y_tokenizer) for p in tqdm(subset_predictions, desc="Decoding sequences")]


Decoding sequences: 100%|██████████| 10/10 [00:00<00:00, 11205.73it/s]


In [51]:
# Decode the reference summaries if they are tokenized
def decode_summary(summary, tokenizer):
    return ' '.join([tokenizer.index_word.get(i, '') for i in summary if i != 0])

# Decode y_test if it's not in string format
decoded_y_test = [decode_summary(summary, y_tokenizer) for summary in y_test]

# Calculate BLEU and ROUGE scores
bleu_scores = [sentence_bleu([ref.split()], pred.split()) for ref, pred in zip(decoded_y_test, predicted_texts)]
rouge_scores = [rouge.get_scores(pred, ref)[0] for ref, pred in zip(decoded_y_test, predicted_texts)]

# Calculate average scores
average_bleu = sum(bleu_scores) / len(bleu_scores)
average_rouge = {
    'rouge-1': np.mean([score['rouge-1']['f'] for score in rouge_scores]),
    'rouge-2': np.mean([score['rouge-2']['f'] for score in rouge_scores]),
    'rouge-l': np.mean([score['rouge-l']['f'] for score in rouge_scores])
}

print("Average BLEU Score:", average_bleu)
print("Average ROUGE Scores:", average_rouge)



Average BLEU Score: 0.0
Average ROUGE Scores: {'rouge-1': 0.0, 'rouge-2': 0.0, 'rouge-l': 0.0}


In [52]:
# Display 10 generated summaries and their corresponding reference summaries
for i in range(10):
    print(f"Generated Summary {i+1}: {predicted_texts[i]}")
    print(f"Reference Summary {i+1}: {decoded_y_test[i]}")
    print("------------------------------------------------\n")


Generated Summary 1: brain migratory paucity shrivel oam oam campari suddard workshop critiqued scheiner rossett brain bushtucker burlap ascend faggots ripping portraying ejf bailiffs bettina manoa dorey spuyten unbound frahn ba813 64km methanethiol mami kokhanok neufield dishong commonwealths fargo graglia misfiled around senora undelivered sidmouth 038 horizontally melborne bridalwear channeling wintak colossi belford columnists poveda dispatcher dispatcher pental harlequins physiological lennar lennar
Reference Summary 1: experts question if packed out planes are putting passengers at risk u s consumer advisory group says minimum space must be stipulated safety tests conducted on planes with more leg room than airlines offer
------------------------------------------------

Generated Summary 2: brain discusses rahulan lundberg mrsa popchella murfy fronsman offenhauser sforza teauneuf sangakarra julienne officious blackman attested commercialize homeschooling gareth surface 141millio

# retrained GRU

In [53]:
!pip install tokenizers


In [54]:
from tokenizers import ByteLevelBPETokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Embedding

# Example dataset
x_train = train_data['cleaned_article']
y_train = train_data['cleaned_highlights']
x_val = validate_data['cleaned_article']
y_val = validate_data['cleaned_highlights']

In [55]:
# Initialize and train the tokenizer
bpe_tokenizer = ByteLevelBPETokenizer()
  # Adjust 'vocab_size' as needed
bpe_tokenizer.train_from_iterator(x_train, vocab_size=30000)


In [56]:




# Tokenize and pad the sequences
x_train_tokenized = [bpe_tokenizer.encode(text).ids for text in x_train]
x_train_padded = pad_sequences(x_train_tokenized, maxlen=800, padding='post')  # Adjust 'maxlen' as needed

# Do the same for y_train
y_train_tokenized = [bpe_tokenizer.encode(text).ids for text in y_train]
y_train_padded = pad_sequences(y_train_tokenized, maxlen=60, padding='post')   # Adjust 'maxlen' as needed

# Ensure that you use the same tokenization and padding processes as your training data
x_val_tokenized = [bpe_tokenizer.encode(text).ids for text in x_val]
x_val_padded = pad_sequences(x_val_tokenized, maxlen=800, padding='post')  # Adjust 'maxlen' as needed

y_val_tokenized = [bpe_tokenizer.encode(text).ids for text in y_val]
y_val_padded = pad_sequences(y_val_tokenized, maxlen=60, padding='post')   # Adjust 'maxlen' as needed



In [57]:
# Vocabulary sizes
x_voc_size = bpe_tokenizer.get_vocab_size()
y_voc_size = x_voc_size  # Assuming same tokenizer for simplicity, adjust as needed

In [58]:
# # Pickle the BPE tokenizer
# with open('bpe_tokenizer.pkl', 'wb') as file:
#     pickle.dump(bpe_tokenizer, file)

# # Pickle the padded training sequences
# with open('x_train_padded.pkl', 'wb') as file:
#     pickle.dump(x_train_padded, file)

# with open('y_train_padded.pkl', 'wb') as file:
#     pickle.dump(y_train_padded, file)


In [59]:
# Pickle the BPE tokenizer
with open('bpe_tokenizer.pkl', 'wb') as file:
    pickle.dump(bpe_tokenizer, file)

# Pickle the padded training sequences
with open('x_val_padded.pkl', 'wb') as file:
    pickle.dump(x_val_padded, file)

with open('y_val_padded.pkl', 'wb') as file:
    pickle.dump(y_val_padded, file)

In [60]:
# # Load the BPE tokenizer
# with open('bpe_tokenizer.pkl', 'rb') as file:
#     bpe_tokenizer = pickle.load(file)

# # Load the padded training sequences
# with open('x_train_padded.pkl', 'rb') as file:
#     x_train_padded = pickle.load(file)

# with open('y_train_padded.pkl', 'rb') as file:
#     y_train_padded = pickle.load(file)


In [61]:
# # Load the BPE tokenizer
# with open('bpe_tokenizer.pkl', 'rb') as file:
#     bpe_tokenizer = pickle.load(file)

# # Load the padded training sequences
# with open('x_val_padded.pkl', 'rb') as file:
#     x_val_padded = pickle.load(file)

# with open('y_val_padded.pkl', 'rb') as file:
#     y_val_padded = pickle.load(file)


In [62]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

def check_data_for_nan_inf(x, y):
    return np.isnan(x).any(), np.isnan(y).any(), np.isinf(x).any(), np.isinf(y).any()

print("Training Data:", check_data_for_nan_inf(x_train_padded, y_train_padded))
print("Validation Data:", check_data_for_nan_inf(x_val_padded, y_val_padded))

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=x_voc_size, output_dim=64)(encoder_inputs)
encoder_gru = GRU(64, return_state=True)
encoder_outputs, encoder_state = encoder_gru(encoder_embedding)

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=y_voc_size, output_dim=64)(decoder_inputs)
decoder_gru = GRU(64, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=encoder_state)
decoder_dense = Dense(y_voc_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

optimizer = Adam(learning_rate=0.001)  # Adjust learning rate as needed
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy')

# Early stopping to monitor the training process
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min')

Training Data: (False, False, False, False)
Validation Data: (False, False, False, False)


In [63]:
checkpoint_path = 'model_checkpoint.h5'  # File path to save the model

checkpoint_callback = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_freq='epoch'  # Save after every epoch
)

In [64]:
history = model.fit(
    [x_train_padded, y_train_padded[:, :-1]],  # Training data: encoder input and decoder input
    y_train_padded.reshape(y_train_padded.shape[0], y_train_padded.shape[1], 1)[:, 1:],  # Training targets
    epochs=5, 
    batch_size=32,
    validation_data=([x_val_padded, y_val_padded[:, :-1]],  # Validation data: encoder input and decoder input
                     y_val_padded.reshape(y_val_padded.shape[0], y_val_padded.shape[1], 1)[:, 1:])  # Validation targets
)


Epoch 1/5
8973/8973 [==============================] - 650s 72ms/step - loss: 5.3065 - val_loss: 4.9617
Epoch 2/5
8973/8973 [==============================] - 449s 50ms/step - loss: 4.6668 - val_loss: 4.7044
Epoch 3/5
8973/8973 [==============================] - 439s 49ms/step - loss: 4.4687 - val_loss: 4.5872
Epoch 4/5
8973/8973 [==============================] - 432s 48ms/step - loss: 4.3593 - val_loss: 4.5200
Epoch 5/5
8973/8973 [==============================] - 430s 48ms/step - loss: 4.2887 - val_loss: 4.4787


In [65]:
# # to resume fromm checkpoint after interruption

# from tensorflow.keras.models import load_model

# # Load the model from the last checkpoint
# model = load_model(checkpoint_path)

# # Continue training
# history = model.fit(
#     [x_train_padded, y_train_padded[:, :-1]],
#     y_train_padded.reshape(y_train_padded.shape[0], y_train_padded.shape[1], 1)[:, 1:], 
#     epochs=remaining_epochs,  # Set the number of remaining epochs
#     batch_size=32,
#     validation_data=(x_val_padded, y_val_padded),
#     callbacks=[checkpoint_callback]
# )


In [66]:
# Save the entire model to a file
model_save_path = 'my_model.h5'  # You can change the path and file name as needed
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to my_model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [67]:
# from tensorflow.keras.models import load_model

# # Load the model
# loaded_model = load_model('my_model.h5')

In [68]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, None, 64)             1920000   ['input_7[0][0]']             
                                                                                                  
 embedding_5 (Embedding)     (None, None, 64)             1920000   ['input_8[0][0]']             
                                                                                            

In [69]:
sample_size = 10 # Try with an even smaller number of samples
x_test_sample = x_val_padded[:sample_size]
y_test_sample = y_val_padded[:sample_size, :-1]

sample_predictions = model.predict([x_test_sample, y_test_sample])

1/1 [==============================] - 1s 616ms/step


In [70]:
from tqdm import tqdm

def decode_sequence(input_sequence, tokenizer):
    # Convert the list of IDs to a string
    token_ids = [int(i) for i in input_sequence if i != 0]  # Skipping padding
    text = tokenizer.decode(token_ids)
    return text

# Assuming 'bpe_tokenizer' is your trained ByteLevelBPETokenizer
# and 'sample_predictions' is the output from your model

subset_size = 100  # Adjust as needed

# Decode a subset of the predictions
subset_predictions = np.argmax(sample_predictions, axis=-1)[:subset_size]
predicted_texts = [decode_sequence(p, bpe_tokenizer) for p in tqdm(subset_predictions, desc="Decoding sequences")]


Decoding sequences: 100%|██████████| 10/10 [00:00<00:00, 11008.67it/s]


In [71]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge import Rouge

def calculate_bleu(reference_texts, predicted_texts):
    smoothing = SmoothingFunction().method1
    bleu_scores = [sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing)
                   for ref, pred in zip(reference_texts, predicted_texts)]
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu

def calculate_rouge(reference_texts, predicted_texts):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_texts, reference_texts, avg=True)
    return scores


In [72]:
# Define your actual reference summaries here
# Decode the reference summaries
decoded_reference_texts = [bpe_tokenizer.decode(ref) for ref in y_test_sample]

reference_texts = y_test_sample  # Replace with your actual data

# Calculate BLEU and ROUGE scores
average_bleu = calculate_bleu(decoded_reference_texts, predicted_texts)
rouge_scores = calculate_rouge(decoded_reference_texts, predicted_texts)

print(f"Average BLEU Score: {average_bleu}")
print(f"ROUGE Scores: {rouge_scores}")



Average BLEU Score: 0.014915843737383602
ROUGE Scores: {'rouge-1': {'r': 0.2162805301213393, 'p': 0.3727255158970223, 'f': 0.27223567928592435}, 'rouge-2': {'r': 0.03373428247683736, 'p': 0.039904539818332925, 'f': 0.03652403269245433}, 'rouge-l': {'r': 0.17369756010078638, 'p': 0.2993304261611884, 'f': 0.21867011881960835}}


In [73]:
# Display 10 generated summaries and their corresponding reference summaries
for i in range(10):
    print(f"Generated Summary {i+1}: {predicted_texts[i]}")
    print(f"Reference Summary {i+1}: {decoded_reference_texts[i]}")
    print("------------------------------------------------\n")


Generated Summary 1: inger the the   the    the    sunday     the  the mother was is  y  the a been the   new francisco moore is the  a ge award the s  ack   the  o   is is in  the in   the  
Reference Summary 1: s and  50s in mgm musicals and films died on march 15   forrest  whose birth name was katherine feeney  had long battled cancer   a san diego native  forrest became a protege of hollywood trailblazer ida lupino  who cast her in starring roles in films
------------------------------------------------

Generated Summary 2:  to a of the palace   ai         the been the out 000 copies of the  the in the year years   the the to from the and the of art    the  
Reference Summary 2: works include pictures of presidential palace and yangtze river bridge   has inked 1 000 pieces of art on leaves in last two years   gives work away to students in form of bookmarks and postcards  !!!!!!!!!!!!!!!!!!
------------------------------------------------

Generated Summary 3:  the a he died   he